In [1]:
import pandas as pd

FILENAME = 'vicuna_benchmark_human_annotations.csv'
df = pd.read_csv(FILENAME)

questions_df = pd.read_json('../../prompts/vicuna_questions.jsonl', lines=True)

In [2]:
questions_df.head()

,question_id,text,category
0,1,How can I improve my time management skills?,generic
1,2,What are the most effective ways to deal with ...,generic
2,3,What are the main differences between Python a...,generic
3,4,How can I increase my productivity while worki...,generic
4,5,Can you explain the basics of quantum computing?,generic


In [3]:
merged = pd.merge(questions_df, df, left_on='text', right_on='prompt', how='outer')
merged.question_id.isna().sum()

0

In [4]:
del merged['prompt']
merged.rename(columns={'answer':'score', 'system_a': 'model_a', 'system_b': 'model_b'}, inplace=True)
merged.score = merged.score.map({'system_a': 1, 'system_b': 2, 'tie': 3})

In [5]:
merged.head(3)
merged

,question_id,text,category,hit_id_hash,worker_id_hash,model_a,model_b,generation_system_a,generation_system_b,score
0,1,How can I improve my time management skills?,generic,b975b20fc9ee76f268564e8c8bf6006e7910b05045495c...,e0d35f0eac2cccefd8102c58b10348edcf1aa17478fd72...,guanaco-7b,guanaco-13b,Time management is a crucial skill that can he...,Time management is a critical skill that can h...,2
1,1,How can I improve my time management skills?,generic,b975b20fc9ee76f268564e8c8bf6006e7910b05045495c...,c27c78603db9a33ac6b748c4f42e670727aa95715b4c0c...,guanaco-7b,guanaco-13b,Time management is a crucial skill that can he...,Time management is a critical skill that can h...,1
2,1,How can I improve my time management skills?,generic,b975b20fc9ee76f268564e8c8bf6006e7910b05045495c...,0a63b300bcaa87fb1bdeed7c7a07c90df618025cb6d527...,guanaco-7b,guanaco-13b,Time management is a crucial skill that can he...,Time management is a critical skill that can h...,1
3,1,How can I improve my time management skills?,generic,d5748cc566a3522b5bdb00e15c08f35f96f423f995f588...,b2b9113195e12c7362c971c1cac2318e26080c985c75e7...,guanaco-13b,gpt4,Time management is a critical skill that can h...,Improving your time management skills can sign...,2
4,1,How can I improve my time management skills?,generic,d5748cc566a3522b5bdb00e15c08f35f96f423f995f588...,c27c78603db9a33ac6b748c4f42e670727aa95715b4c0c...,guanaco-13b,gpt4,Time management is a critical skill that can h...,Improving your time management skills can sign...,3
...,...,...,...,...,...,...,...,...,...,...
6715,80,"Write a symphony concert review, discussing th...",writing,810d1c74aca0f04b44dc91bdfc0d5909994ca550321fa4...,9157445dcc1dedb1107c34d4d0fa9b6494af7c578a14b3...,guanaco-65b,gpt35,Sure! Here's a sample review of a symphony con...,I'm happy to assist you in writing a symphony ...,2
6716,80,"Write a symphony concert review, discussing th...",writing,810d1c74aca0f04b44dc91bdfc0d5909994ca550321fa4...,31f00f48b0a0579b5823922088a8b3bbdd4f24719c5c53...,guanaco-65b,gpt35,Sure! Here's a sample review of a symphony con...,I'm happy to assist you in writing a symphony ...,3
6717,80,"Write a symphony concert review, discussing th...",writing,3ce4453a4ff9311cba017b0e2a316490db88d5d41374ab...,93914b9d066cd283f7530d12c3e15ac5e14acd37a7c3d6...,gpt35,vicuna-13b,I'm happy to assist you in writing a symphony ...,"The concert was a true spectacle of sound, fea...",1
6718,80,"Write a symphony concert review, discussing th...",writing,3ce4453a4ff9311cba017b0e2a316490db88d5d41374ab...,0a63b300bcaa87fb1bdeed7c7a07c90df618025cb6d527...,gpt35,vicuna-13b,I'm happy to assist you in writing a symphony ...,"The concert was a true spectacle of sound, fea...",1


In [6]:
# get list of models in merged
modelsA = merged.model_a.unique()
modelsB = merged.model_b.unique()

for modelA in modelsA:
    for modelB in modelsB:
        if modelA == modelB:
            continue
        filtered_df = merged[(merged['model_a'] == modelA) & (merged['model_b'] == modelB)]
        
        filtered_df = filtered_df[['question_id', 'text', 'score']]

        name = f'answer_{modelA}-vs-answer_{modelB}-human-reviewer_threeclass.jsonl'
        filtered_df.to_json(f'../{name}', orient='records', lines=True)